<a href="https://colab.research.google.com/github/GirumSe/iCog-Internship-Tasks/blob/main/RAG_Task2_by_Girum_Senay.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Installing and Preparing the Packages needed

In [ ]:
!pip install accelerate langchain bitsandbytes -qq
!pip install pinecone-client -qq
!pip install llama_index sentence_transformers -qq
!pip install langchain-community -qq
!pip install together -qq
!pip install neo4j -qq
!pip install yfiles_jupyter_graphs -qq

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.4/50.4 kB 831.2 kB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 997.8/997.8 kB 10.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 137.5/137.5 MB 5.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 391.5/391.5 kB 15.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 140.4/140.4 kB 8.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 141.9/141.9 kB 10.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 244.8/244.8 kB 5.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 117.6/117.6 kB 8.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 227.1/227.1 kB 6.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 15.5/15.5 MB 79.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 50.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 169.4/169.4 kB 11.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [ ]:
from google.colab import userdata
uri = userdata.get('NEO4J_URI')
user = userdata.get('NEO4J_USERNAME')
password = userdata.get('NEO4J_PASSWORD')
together_key = userdata.get('TOGETHER_API_KEY')
pinecone_key= userdata.get('PINECONE_API_KEY')

#Neo4j

##Querying

In [ ]:
from neo4j import GraphDatabase
driver = GraphDatabase.driver(uri, auth=(user, password))

In [ ]:
from together import Together
import re
client = Together(api_key=together_key)

###Query and Other utility Function

In [ ]:
from google.colab import output
output.enable_custom_widget_manager()

In [ ]:
from yfiles_jupyter_graphs import GraphWidget
def run_query(query, parameters=None):
    try:
        with driver.session() as session:
            results = session.run(query, parameters)
            graph = session.run(query, parameters).graph()
            records = [dict(record) for record in results]
            if not records:
                all_data_query = "MATCH (n)-[r]->(m) RETURN n, r, m"
                all_results = session.run(all_data_query)
                graph = session.run(all_data_query).graph()
                records = [dict(record) for record in all_results]
                ygraph = GraphWidget(graph = graph)
                display(ygraph)
                return records
            ygraph = GraphWidget(graph = graph)
            display(ygraph)
            return records
    except Exception as e:
        print(f"An error occurred: {e}")
        with driver.session() as session:
            all_data_query = "MATCH (n)-[r]->(m) RETURN n, r, m"
            all_results = session.run(all_data_query)
            graph = session.run(all_data_query).graph()
            records = [dict(record) for record in all_results]
            ygraph = GraphWidget(graph = graph)
            display(ygraph)
            return records

In [ ]:
# query = "MATCH (n) RETURN n"
query = """
MATCH (n)-[r]->(m)
RETURN n, r, m
LIMIT 10
"""
result = run_query(query)

GraphWidget(layout=Layout(height='500px', width='100%'))

In [ ]:
def nl_to_neoquery(question):
    context = """
    Entities:
    - PLAYER: {name, age, number, height, weight}
    - COACH: {name}
    - TEAM: {name}

    Relationships:
    - TEAMMATES: Bidirectional between PLAYER nodes
    - COACHES: From COACH to PLAYER
    - PLAYS_FOR: From PLAYER to TEAM with a salary property
    - COACHES_FOR: From COACH to TEAM
    - PLAYED_AGAINST: From PLAYER to TEAM with properties {minutes, points, assists, rebounds, turnovers}

    Query requirements:
    - When matching names (e.g., PLAYER, COACH, or TEAM names), ensure the query is case-insensitive by using `toLower()` function on both the property and the search string.
    - Support partial name searches by using the `CONTAINS` operator.
    - Many times just try to get the whole information of players when asked about single player, teams when asked about single team, and coaches when asked about single coach.
    - If you want to get match statistics:
      - Retrieve each PLAYER's name, individual game stats (match_stats), and total stats (total_stat).
      - The query should return results :
        - `Name` as the player's name.
        - `match_stats` as a list of maps, with each map containing `against`, `points`, `assists`, `rebounds`, `turnovers`, and `minutes`.
        - `total_stat` as a map containing the total points, assists, rebounds, turnovers, and minutes.
      - Ensure the query uses a `WITH` clause to properly aggregate statistics and group data before returning it if you use it..
      - Use the `WITH` clause to alias the player's name and aggregate statistics if you use it.
      - Use `sum` in r.[stat_properties name] when calculating the total.
    - The final Cypher query should be valid, executable, and avoid any aggregation errors.
    """


    # Construct the prompt
    prompt = f"""
    Database Context:
    {context}

    Question:
    {question}

    Generate a Cypher query that can be used to extract the data from the above database, that can be helpful to you to answer the question asked.
    return only the query in square brackets and don't format it and don't comment on anything else,!
    """
    stream = client.chat.completions.create(
    model="meta-llama/Meta-Llama-3.1-70B-Instruct-Turbo",
    messages=[{"role": "user", "content": prompt}],
    stream=True,
    )
    query = ""
    for chunk in stream:
      query += chunk.choices[0].delta.content or ""
      #print(chunk.choices[0].delta.content or "", end="", flush=True)

    clean_text = re.sub(r'^\[|\]$', '', query)
    return clean_text

#Adding to the Pincone Database

###preparing embadding model

In [ ]:
from langchain.embeddings import HuggingFaceEmbeddings, SentenceTransformerEmbeddings
from llama_index.legacy.embeddings.langchain import LangchainEmbedding

embed_model = LangchainEmbedding(
  HuggingFaceEmbeddings(model_name="sentence-transformers/all-mpnet-base-v2")
)

[nltk_data] Downloading package stopwords to
[nltk_data]     /usr/local/lib/python3.10/dist-
[nltk_data]     packages/llama_index/legacy/_static/nltk_cache...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt to
[nltk_data]     /usr/local/lib/python3.10/dist-
[nltk_data]     packages/llama_index/legacy/_static/nltk_cache...
[nltk_data]   Unzipping tokenizers/punkt.zip.
/usr/local/lib/python3.10/dist-packages/langchain_core/_api/deprecation.py:141: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 0.3.0. An updated version of the class exists in the langchain-huggingface package and should be used instead. To use it run `pip install -U langchain-huggingface` and import as `from langchain_huggingface import HuggingFaceEmbeddings`.
  warn_deprecated(
/usr/local/lib/python3.10/dist-packages/sentence_transformers/cross_encoder/CrossEncoder.py:11: TqdmExperimentalWarning: Using `tqdm.auto

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.6k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/363 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

### Connect to Pinecone to store the vectors

In [ ]:
from pinecone import Pinecone, ServerlessSpec
pinecone = Pinecone(
        api_key=pinecone_key)

In [ ]:
index_name = 'basketfacts'
pinecone_index = pinecone.Index(index_name)

In [ ]:
def get_data_from_db(query):
    xq = embed_model.get_text_embedding(query)
    result = pinecone_index.query(vector=xq, top_k=20, includeMetadata=True)
    matches = []
    for i in result['matches']:
      if (i['score'] > 0.3):
        # append best results
        matches.append(i.metadata['text'])
    return str(matches)

In [ ]:
DEFAULT_SYSTEM_PROMPT = """
GENERAL INSTRUCTIONS:

You will be provided with factual sentences about last season's basketball events. Additionally, data fetched from a Neo4j database may be provided if available.

- Base your answers on the data given.
- Give Data more weight if it is consise and not bulky.
- Display the nodes and edges as proof alongside the answer in natural language format.
- If there isn't enough context or data, simply reply with "I don't have enough data to tell you that."

"""

def get_prompt(instruction):
    prompt_template = DEFAULT_SYSTEM_PROMPT + instruction + "\n"
    return prompt_template

In [ ]:
def format_prompt(query, facts, data):
    return '''
    ### facts:
    {facts}
    ### Data:
    {data}
    based on this data
    ### User Question:
    {query}
    '''.format(facts=facts, query=query, data=data)

In [ ]:
def answer(query):
    facts = get_data_from_db(query)
    data = run_query(nl_to_neoquery(query))
    prompt = format_prompt(query, facts, data)
    prompt_template = get_prompt(prompt)
    stream = client.chat.completions.create(
    model="meta-llama/Meta-Llama-3.1-70B-Instruct-Turbo",
    messages=[{"role": "user", "content": prompt_template}],
    stream=True,
    )

    for chunk in stream:
        print(chunk.choices[0].delta.content or "", end="", flush=True)

##Prompting

Prompts that i tested are:

*   "who played the most minutes in a single game?"
*   "which player have the most turnovers in a single game?"
*   "Who coaches Brooklyn Nets?"

In [ ]:
answer("who played the most minutes in a single game?")

GraphWidget(layout=Layout(height='500px', width='100%'))

Based on the provided data, James Harden played 46 minutes in a single game against Memphis Grizzlies.

Node: James Harden
Edge: played against Memphis Grizzlies
Property: minutes = 46

However, I also found another fact that mentions LeBron James played 38 minutes and 23 minutes in two separate games against Memphis Grizzlies, and 32 minutes against Philadelphia 76ers. But none of these minutes are more than 46 minutes.

Additionally, I found another fact that mentions Giannis Antetokounmpo played 45 minutes against Philadelphia 76ers, which is close to 46 minutes but still less.

Therefore, based on the provided data, James Harden played the most minutes in a single game, which is 46 minutes.